In [ ]:
import os
import glob

import cv2
import pathlib
import traceback

from tqdm import tqdm

In [ ]:
def resize_image(im, max_side_len=1200):
    """
    resize image to a size multiple of 32 which is required by the network
    :param im: the resized image
    :param max_side_len: limit of max image size to avoid out of memory in gpu
    :return: the resized image and the resize ratio
    """
    h, w, _ = im.shape

    resize_w = w
    resize_h = h

    # limit the max side
    if max(resize_h, resize_w) > max_side_len:
        ratio = float(max_side_len) / resize_h if resize_h > resize_w else float(max_side_len) / resize_w
    else:
        ratio = 1.
    resize_h = int(resize_h * ratio)
    resize_w = int(resize_w * ratio)

    resize_h = resize_h if resize_h % 32 == 0 else (resize_h // 32 - 1) * 32
    resize_w = resize_w if resize_w % 32 == 0 else (resize_w // 32 - 1) * 32
    im = cv2.resize(im, (int(resize_w), int(resize_h)))

    ratio_h = resize_h / float(h)
    ratio_w = resize_w / float(w)

    return im, (ratio_h, ratio_w)

In [ ]:
def process_txt(file_names, gt_path, new_gt_path, ratio_dict):
    for file_name in file_names[1:]:
        src = os.path.join(gt_path, file_name)
        with open(src, 'r', encoding='utf-8') as f:
            new_texts = ''
            lines = f.readlines()
            for line in lines:
                texts = line[:-1].split(',')
                h = ratio_dict[file_name[7:-4]][0]
                w = ratio_dict[file_name[7:-4]][1]
                bbox = (f'{int(int(texts[0])*w)}, {int(int(texts[1])*h)}, {int(int(texts[2])*w)}, {int(int(texts[3])*h)}, {int(int(texts[4])*w)}, {int(int(texts[5])*h)}, {int(int(texts[6])*w)}, {int(int(texts[7])*h)}')
                
                script = texts[8]
                word = texts[9]
                
                new_texts += bbox+','+script+','+word+'\n'

        new_src = os.path.join(new_gt_path, file_name)
        with open(new_src,'w') as f:
            f.write(new_texts)
            
    print('success_make_resized_gts')

In [ ]:
def process_image(image_path, new_image_path, max_side_size):
    types = ('*.jpg', '*.png', '*.JPG', '*.PNG')  # the tuple of file types
    files_grabbed = []
    ratio_dict = {}
    for files in types:
        files_grabbed.extend(image_path.glob(files))
    for image_fn in tqdm(files_grabbed):
        try:
            im = cv2.imread(image_fn.as_posix())[:, :, ::-1]
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            im_resized, ratio_dict[image_fn.as_posix()[len(image_path.as_posix())-8:-4]] = resize_image(im, max_side_size)
            cv2.imwrite(new_image_path+'/'+image_fn.name, im_resized)
        except Exception as e:
            print('excepted')
            traceback.print_exc()
            print(image_fn)
    print('success_saving_resized_images')
            
    return ratio_dict

In [ ]:
# 이미지와 gt의 위치
datasets_path = '/opt/ml/final_project/final-project-level3-cv-07/models/datasets'
image_path = datasets_path + '/' + 'test_images'
gt_path = datasets_path + '/' + 'test_gts'

# 파일 이름 추출
file_names = sorted(os.listdir(gt_path))

# 저장할 폴더 이름
new_gt_path = gt_path + '_resized'
new_image_path = image_path + '_resized'

# 리사이징할 크기
max_side_size=1200

if not os.path.isdir(new_gt_path):
    os.makedirs(new_gt_path)
if not os.path.isdir(new_image_path):
    os.makedirs(new_image_path)

ratio_dict = process_image(pathlib.Path(image_path), new_image_path, max_side_size)
process_txt(file_names, gt_path, new_gt_path, ratio_dict)